Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2019.1

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de carga de trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [1]:
using Distributed


addprocs(3)

@everywhere begin
    workloads = fill(0.0, nworkers()+1)
    leader = nworkers()+1
    candidate = -1
end


#= 
    it updates the reference for the new leader and verifies that 
    the current process is the master to print the message. 
=#
@everywhere function update_leader(new_leader)
    my_id = Distributed.myid()
    
    if my_id == 1
        println("New leader elected! Worker ", new_leader)
    end

    global leader = new_leader
    
end


#=
    If the current proc is the one that starts the election and the current 
    leader is not the best candidate, the current one is the new leader.
    In case of a tie, the one with the highest ID is chosen.
    If the current one has a lower workload, it will be the new best candidate.
=#
@everywhere function vote(elector, start, workload)
    
    my_id = Distributed.myid()
    myworkload = workloads[my_id]    
    next = my_id == nprocs() ? 2 : (my_id + 1)

    if start == my_id && leader != elector
        for id in procs()
            @spawnat id update_leader(elector)
        end
        
    elseif workload == myworkload
        
        if elector > my_id 
            global candidate = elector 
            @spawnat next vote(candidate, start, workload)
        else
            global candidate = my_id 
            @spawnat next vote(candidate, start, myworkload)
        end 
        
    elseif myworkload < workload
        global candidate = my_id
        @spawnat next vote(candidate, start, myworkload)
    
    else
        global candidate = elector 
        @spawnat next vote(candidate, start, workload)
    end
    
end


#=
    In this function, the first candidate of the election is defined. 
    The election continues with the vote of the next individual.
=#
@everywhere function start_election(worker, workload)

    global candidate = worker
    next = worker == nprocs() ? 2 : (worker + 1)
    @spawnat next vote(candidate, worker, workload)
    
end


#=
    If the current node is the Master, then the workloads are displayed. 
    In this function, the conditions necessary for a new election to be started are verified.
=#
@everywhere function work(id, workload_aux)
    my_id = Distributed.myid()
    workloads[id] = workload_aux    
    
    if my_id == 1
        println("Workload from Worker ", id, ": ", workloads[2:nworkers()+1])
    
    elseif id == leader && workload_aux >= 0.8 && workloads[my_id] < 0.8
        start_election(my_id, workloads[my_id]) 
    end
    
end


@everywhere function main()
    my_id = Distributed.myid()
    
    while true
        sleep(2)
        
        workload = round(rand(1)[1], digits = 1)
        for proc in procs()                   
            @spawnat proc work(my_id, workload)
        end
        
    end
end


println("Initial Leader: ", leader)
for id in workers()
    @spawnat id main()
end

Initial Leader: 4
Workload from Worker 2: [0.4, 0.0, 0.0]
Workload from Worker 3: [0.4, 0.4, 0.0]
Workload from Worker 4: [0.4, 0.4, 0.3]
Workload from Worker 4: [0.4, 0.4, 1.0]
Workload from Worker 3: [0.4, 0.0, 1.0]
Workload from Worker 2: [0.1, 0.0, 1.0]
New leader elected! Worker 3
New leader elected! Worker 3
Workload from Worker 4: [0.1, 0.0, 0.9]
Workload from Worker 3: [0.1, 0.9, 0.9]
New leader elected! Worker 2
Workload from Worker 2: [0.1, 0.9, 0.9]
Workload from Worker 4: [0.1, 0.9, 0.6]
Workload from Worker 3: [0.1, 0.9, 0.6]
Workload from Worker 2: [0.9, 0.9, 0.6]
New leader elected! Worker 4
Workload from Worker 4: [0.9, 0.9, 0.8]
Workload from Worker 3: [0.9, 0.1, 0.8]
Workload from Worker 2: [0.9, 0.1, 0.8]
Workload from Worker 4: [0.9, 0.1, 0.9]
New leader elected! Worker 3
Workload from Worker 3: [0.9, 0.7, 0.9]
Workload from Worker 2: [0.5, 0.7, 0.9]
Workload from Worker 4: [0.5, 0.7, 0.3]
Workload from Worker 3: [0.5, 0.1, 0.3]
Workload from Worker 2: [0.7, 0.1, 0.


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.